Sales Probability: Provide an estimated number of days to which the property can be sold

In [97]:
#importing modules
import tensorflow as tf
from tensorflow import keras
from keras import datasets
from keras import layers
from keras import models
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as K
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [98]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [99]:
data_path = "data.csv"
df = pd.read_csv(data_path)
df.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [100]:
state_df = df[df['state'] == 'New Jersey']

In [101]:
state_df_duplicate = pd.concat([state_df,state_df,state_df,state_df,state_df,state_df,state_df,state_df], ignore_index=True)


In [102]:
state_df_cleaned= state_df_duplicate.dropna()

In [103]:
state_df_cleaned.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
1,16709.0,for_sale,339900.0,2.0,2.0,0.11,893593.0,Avenel,New Jersey,7001.0,1456.0,1987-05-20
2,60594.0,for_sale,305100.0,3.0,2.0,0.14,169015.0,Avenel,New Jersey,7001.0,1542.0,2022-01-27
5,53550.0,for_sale,325000.0,3.0,3.0,0.15,1354688.0,Avenel,New Jersey,7001.0,1870.0,2002-12-17
8,81259.0,for_sale,440000.0,4.0,2.0,0.17,863208.0,Avenel,New Jersey,7001.0,1842.0,2015-11-23
10,48366.0,for_sale,399000.0,3.0,3.0,0.06,190536.0,Avenel,New Jersey,7001.0,1476.0,1990-02-28


In [104]:
set(state_df_cleaned['status'])

{'for_sale', 'sold'}

In [105]:
sold_df = state_df_cleaned[state_df_cleaned['status'] == 'sold']

In [106]:
sold_df.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
33563,81259.0,sold,440000.0,4.0,2.0,0.17,863208.0,Avenel,New Jersey,7001.0,1842.0,2022-04-11
33566,53016.0,sold,385000.0,3.0,2.0,0.11,531675.0,Avenel,New Jersey,7001.0,968.0,2022-04-01
33567,51784.0,sold,479000.0,3.0,2.0,0.05,1089635.0,Bayonne,New Jersey,7002.0,1322.0,2022-04-12
33569,8149.0,sold,729000.0,5.0,3.0,0.10,893972.0,Bayonne,New Jersey,7002.0,2089.0,2022-03-07
33570,79122.0,sold,715000.0,6.0,5.0,0.06,1746025.0,Bayonne,New Jersey,7002.0,2500.0,2022-03-07


In [107]:
set(sold_df['status'])

{'sold'}

In [108]:
sold_df.drop(['status','brokered_by'], axis=1, inplace=True)

/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_51708/2220761013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sold_df.drop(['status','brokered_by'], axis=1, inplace=True)


In [109]:
sold_df.columns

Index(['price', 'bed', 'bath', 'acre_lot', 'street', 'city', 'state',
       'zip_code', 'house_size', 'prev_sold_date'],
      dtype='object')

In [110]:
sold_df['prev_sold_date'] = pd.to_datetime(sold_df['prev_sold_date'])

/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_51708/499865651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sold_df['prev_sold_date'] = pd.to_datetime(sold_df['prev_sold_date'])


In [111]:
sold_df.dropna(inplace=True)

/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_51708/914261620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sold_df.dropna(inplace=True)


In [112]:
sold_df['prev_sold_date']

33563    2022-04-11
33566    2022-04-01
33567    2022-04-12
33569    2022-03-07
33570    2022-03-07
            ...    
385587   2021-12-30
385588   2022-01-21
385589   2021-11-17
385590   2022-02-01
385591   2022-02-15
Name: prev_sold_date, Length: 53472, dtype: datetime64[ns]

In [113]:
#adding a new column 'previous_listed_date' to the dataframe
sold_df=sold_df.assign(prev_listed_date = sold_df['prev_sold_date'].shift(1))  

In [114]:
sold_df['prev_listed_date']

33563           NaT
33566    2022-04-11
33567    2022-04-01
33569    2022-04-12
33570    2022-03-07
            ...    
385587   2021-10-28
385588   2021-12-30
385589   2022-01-21
385590   2021-11-17
385591   2022-02-01
Name: prev_listed_date, Length: 53472, dtype: datetime64[ns]

In [115]:
#here we add the number of days the property was listed for
sold_df['days_listed'] = sold_df['prev_sold_date'] - sold_df['prev_listed_date']

In [116]:
sold_df['days_listed']

33563         NaT
33566    -10 days
33567     11 days
33569    -36 days
33570      0 days
           ...   
385587    63 days
385588    22 days
385589   -65 days
385590    76 days
385591    14 days
Name: days_listed, Length: 53472, dtype: timedelta64[ns]

In [117]:
X=sold_df.drop(['days_listed','prev_sold_date','prev_listed_date'], axis=1)
y=pd.to_numeric(sold_df['days_listed'])

In [118]:
X.columns

Index(['price', 'bed', 'bath', 'acre_lot', 'street', 'city', 'state',
       'zip_code', 'house_size'],
      dtype='object')

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
numerical_features = ['bed', 'bath', 'acre_lot', 'house_size','zip_code','street','price']
categorical_features = ['city', 'state']
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
X_train_preprocessed = pipeline.fit_transform(X_train)
X_test_preprocessed = pipeline.transform(X_test)

In [120]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_preprocessed.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Output layer with a single neuron for regression

model.compile(optimizer='adam', loss='mean_squared_error')



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [121]:

history = model.fit(X_train_preprocessed, y_train, epochs=5, validation_split=0.2)

Epoch 1/5
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 706us/step - loss: 657508640604815360.0000 - val_loss: 666524223635718144.0000
Epoch 2/5
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 592us/step - loss: 671058678308143104.0000 - val_loss: 666523192843567104.0000
Epoch 3/5
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 571us/step - loss: 669658106652786688.0000 - val_loss: 666521474856648704.0000
Epoch 4/5
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 533us/step - loss: 661973207569399808.0000 - val_loss: 666517489126998016.0000
Epoch 5/5
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1s 536us/step - loss: 668732317862199296.0000 - val_loss: 666512678763626496.0000


In [122]:
loss = model.evaluate(X_test_preprocessed, y_test)
print(f'Test Loss: {loss}')


335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 290us/step - loss: 1204857447855247763439616.0000
Test Loss: 2.0241593825337417e+26


In [123]:
# Make predictions on new data
new_data = pd.DataFrame({
    'bed': [4],
    'bath': [3],
    'acre_lot': [0.8],
    'street': [893593.0],
    'city': ['Newark'],
    'state': ['New Jersey'],
    'house_size': [2200],
    'zip_code': [7003],
    'price': [500000]
})

# Preprocess the new data
new_data_preprocessed = pipeline.transform(new_data)

# Predict the price
predicted_price = model.predict(new_data_preprocessed)
print(f'Predicted Price: {predicted_price[0][0]:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted Price: -90078.76


In [125]:
model.save('sales_probability_prediction_model.keras')